<img src="logo.png">

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [14, 14]
np.random.seed(42)

# Importamos los datos

In [ ]:
vehiculos = pd.read_csv("vehiculos_procesado_con_grupos.csv").drop(
            ["fabricante", "modelo", "transmision", "traccion", "clase", "combustible", "consumo"], 
    axis=1)

In [ ]:
vehiculos.head()

In [ ]:
datos_numericos = vehiculos.select_dtypes([int, float])
datos_categoricos = vehiculos.select_dtypes([object, "category"])

In [ ]:
datos_numericos

In [ ]:
for col in datos_numericos.columns:
    datos_numericos[col].fillna(datos_numericos[col].mean(), inplace=True)

Un aspecto importante a tener en cuenta cuando usamos Kmedias es que las distancias dependen de las escalas de las variables. Por lo tanto, es conveniente normalizar datos antes de continuar

In [ ]:
from sklearn.preprocessing import MinMaxScaler

datos_numericos_normalizado = MinMaxScaler().fit_transform(datos_numericos)
datos_numericos_normalizado = pd.DataFrame(datos_numericos_normalizado,
                                               columns=datos_numericos.columns) 

In [ ]:
datos_categoricos_codificados = pd.get_dummies(datos_categoricos, drop_first=True)

In [ ]:
vehiculos_procesado = pd.concat([datos_numericos_normalizado, datos_categoricos_codificados], axis=1)

In [ ]:
vehiculos_procesado.shape

In [ ]:
vehiculos_procesado.head()

# K-Medias

Scikit-learn implementa KMedias en el módulo [sklearn.cluster.KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
estimador_kmedias = KMeans(random_state=42, n_clusters=8)

estimador_kmedias.fit(vehiculos_procesado)

Dado que es aprendizaje supervisado, nos interesan las clases (los clusters) de los datos de entrenamiento.

In [ ]:
clusters = estimador_kmedias.labels_
clusters

No obstante, como cualquier estimador, podemos usarlo para asignar clusters a nuevos elementos.

In [ ]:
estimador_kmedias.predict(vehiculos_procesado)

In [ ]:
centroides = estimador_kmedias.cluster_centers_
centroides

In [ ]:
centroides.shape

Podemos ver también la inercia final de los clusters

In [ ]:
estimador_kmedias.inertia_

In [ ]:
print(KMeans.__doc__)

Los hiperparámetros más importantes del algoritmo `KMeans`:

- **n_clusters**: El número de clusters a crear, o sea **K**. Por defecto es 8
- **init**: Método de inicialización. Un problema que tiene el algoritmo K-Medias es que la solucción alcanzada varia según la inicialización de los centroides. `sklearn` empieza usando el método `kmeans++` que es una versión más moderna y que proporciona mejores resultados que la inicialización aleatoria (random)
- **n_init**: El número de inicializaciones a probar. Básicamente `KMeans` aplica el algoritmo `n_init` veces y elige los clusters que minimizan la inercia.
- **max_iter**: Máximo número de iteraciones para llegar al criterio de parada.
- **tol**: Tolerancia para declarar criterio de parada (cuanto más grande, antes parará el algoritmo).



In [ ]:
from sklearn.metrics import euclidean_distances

In [ ]:
distancias_centroides = euclidean_distances(centroides)
distancias_centroides

In [ ]:
list(zip(np.argmax(distancias_centroides, axis=1), np.max(distancias_centroides, axis=1)))

In [ ]:
def resumen_cluster(cluster_id):
    cluster = vehiculos[clusters==cluster_id]
    resumen_cluster = cluster[datos_categoricos.columns].mode().to_dict(orient="records")[0]
    resumen_cluster.update(cluster.mean().to_dict())
    resumen_cluster["cluster_id"] = cluster_id
    return resumen_cluster

def comparar_clusters(*cluster_ids):
    resumenes = []
    for cluster_id in cluster_ids:
        resumenes.append(resumen_cluster(cluster_id))
    return pd.DataFrame(resumenes).set_index("cluster_id").T

In [ ]:
resumen_cluster(0)

In [ ]:
comparar_clusters(0,5)

In [ ]:
comparar_clusters(*np.unique(clusters))

En el Análisis exploratorio de datos a veces es importante incluir un clustering para indicar posibles grupos naturales en el dataset. 

In [ ]:
def kmeans_cluster(df, n_clusters=2):
    model = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = model.fit_predict(df)
    cluster_results = df.copy()
    cluster_results['Cluster'] = clusters
    return cluster_results

def resumen_grafico_clustering(results):
    cluster_size = results.groupby(['Cluster']).size().reset_index()
    cluster_size.columns = ['Cluster', 'Count']
    cluster_means = results.groupby(['Cluster'], as_index=False).mean()
    cluster_summary = pd.merge(cluster_size, cluster_means, on='Cluster')
    cluster_summary = cluster_summary.drop(["Count"], axis=1).set_index("Cluster")
    return cluster_summary[sorted(cluster_summary.columns)]

In [ ]:
cluster_results = kmeans_cluster(vehiculos_procesado, 8)
cluster_summary = resumen_grafico_clustering(cluster_results)

In [ ]:
cluster_summary

In [ ]:
import seaborn as sns
matplotlib.rcParams['figure.figsize'] = [14, 14]

In [ ]:
sns.heatmap(cluster_summary.transpose(), annot=True);

# ¿Cómo elegir K?

Hay varias opciones para elegir K

**1.Conocimiento de dominio**
A veces es posible tomar una decisión razonable a priori respecto al número de clusters que queremos. Por ejemplo, supongamos que queremos agrupar un conjunto de películas. Un valor razonable de K sería el número de categorías de películas en IMDB.

**2. Decisión de negocio**
Hay veces que la decisión del número de clusters viene dada por el negocio. Por ejemplo, supongamos que estamos agrupando un conjunto de invitados a un banquete. En ese caso el valor de K vendría dado por el número disponible de mesas.

**3.Método del codo [elbow method](https://en.wikipedia.org/wiki/Elbow_method_(clustering)))**
El método del codo usa como métrica el porcentaje de la varianza explicado como factor respecto al número de clusters. Se intenta buscar aquel número de clusters donde el añadir un cluster más no aumente demasiado dicho porcentaje (es decir, el "codo" de la gráfica que representa esto implica llegar al punto de ganancias decrecientes, donde añadir un cluster nuevo no reduce la varianza de forma significativa.
El porcentaje de la varianza se representa como la variance entre grupos dividida de la varianza total

In [ ]:
from scipy.spatial.distance import cdist

In [ ]:
print(cdist.__doc__)

In [ ]:
varianza_total = cdist(XA=vehiculos_procesado, XB=np.array([vehiculos_procesado.mean()]))

In [ ]:
suma_varianza_total = varianza_total.sum()

In [ ]:
suma_varianza_total

Ahora creamos funciones para calcular varianza intra cluster (wss) y la medida de varianza explicada (definida como la reduccion de la varianza en porcentaje respecto a la varianza máxima (que sería la varianza para k=1).

In [ ]:
def varianza_cluster(cluster_id, centroide_cluster, etiquetas_clusters):
    elementos_cluster = vehiculos_procesado[etiquetas_clusters==cluster_id]
    return cdist(XA=elementos_cluster, XB=np.array([centroide_cluster])).sum()

def medida_varianza(estimador_kmedias, suma_varianza_total):
    etiquetas_clusters = estimador_kmedias.labels_
    wss = 0
    for i, cluster_id in enumerate(np.unique(etiquetas_clusters)):
        centroide_cluster = estimador_kmedias.cluster_centers_[i]
        wss += varianza_cluster(cluster_id, centroide_cluster, etiquetas_clusters)
    return (suma_varianza_total-wss) / suma_varianza_total

Creamos ahora otra medida de evaluación que simplemente usa la inercia

In [ ]:
def medida_inercia(estimador_kmedias):
    return estimador_kmedias.inertia_

Ahora creamos una funcion que evalue para un valor de k las dos métricas

In [ ]:
def evaluar_k_kmedias(k, medida, **kwargs):
    if medida=="inercia":
        funcion_medida = medida_inercia
    elif medida=="varianza":
        funcion_medida = medida_varianza
        
    estimador_kmedias = KMeans(random_state=42, n_clusters=k)
    estimador_kmedias.fit(vehiculos_procesado)
    return funcion_medida(estimador_kmedias, **kwargs)

In [ ]:
resultados_k = {}
rango_k = [5, 10, 20, 30, 50, 75, 100, 200, 300]
for k in rango_k:
    resultados_k[k] = evaluar_k_kmedias(k, 
                                "inercia"), evaluar_k_kmedias(k, "varianza", 
                                                              suma_varianza_total=suma_varianza_total)

In [ ]:
resultados_k

Ahora hacemos una gráfica para ver donde está el "codo" de forma aproximada.

In [ ]:
fig, ax1 = plt.subplots()

ax1.plot(
    [c[0] for c in resultados_k.items()],
    [c[1][0] for c in resultados_k.items()], label="inercia", color="red")
ax1.set_ylabel('inercia')

ax2 = ax1.twinx()
ax2.plot(
    [c[0] for c in resultados_k.items()],
    [c[1][1] for c in resultados_k.items()], label="varianza")
ax2.set_ylabel('varianza explicada')

plt.xlabel("K")
plt.legend()
plt.title("Variación de inercia/varianza respecto a K");

Para este caso en concreto, un valor de K=100 o 150, podria ser una buena opción si no hubiese ningún otro requisito.